
# Tensor-based analysis of Elliptic dataset (DEMO) - Ethan Pabbathi
-----

## 1\. What is a Tensor? (The Data Cube)

### Mathematical Theory

At its core, a tensor is a multi-dimensional array of numbers. You're already familiar with the first two orders:

  * A **1st-order tensor** is a **vector** (a single list of numbers).
  * A **2nd-order tensor** is a **matrix** (a grid of numbers, like a spreadsheet).
  * A **3rd-order tensor**, which we use in this project, is like a **cube of numbers**.

In our project, we model the transaction data as a 3rd-order tensor where the dimensions are **(Senders, Receivers, Time)**. Each cell `T[i, j, k]` in this cube represents the transaction amount from sender `i` to receiver `j` at time `k`.

This structure is powerful because it preserves the complete relationship between entities, unlike a simple 2D matrix which would force us to lose the time dimension. The concept of **tensor rank** is the smallest number of simple, rank-1 tensors that can be summed up to produce the original tensor. This idea is central to decomposition.

### How it Relates to the Code

In the script, this is the first thing we do:

```python
# Create an empty 3D tensor (our "data cube")
tensor = np.zeros((num_senders, num_receivers, num_timesteps))

# Plant patterns by assigning values to specific T[i, j, k] coordinates
tensor[8, 9, 3] = 100.0
```

We build a 3D `numpy` array and "plant" patterns by placing values at specific `(sender, receiver, time)` coordinates. This simulates the real-world data structure.

-----

## 2\. Tucker Decomposition (High-Level Compression)

### Mathematical Theory

Tucker decomposition is a method for compressing a tensor into a smaller **core tensor** and a set of **factor matrices**, one for each dimension. Think of it as a more powerful version of Principal Component Analysis (PCA) for multi-dimensional data.

The formula is:
$X \approx G \times_1 A \times_2 B \times_3 C$

Where:

  * $X$ is the original, large tensor.
  * $G$ is the small, compressed core tensor. It captures the high-level interactions between the groups found in the factor matrices.
  * $A, B, C$ are the factor matrices for senders, receivers, and time, respectively. They act as "dictionaries" that describe the archetypal groups within each dimension.

### How it Relates to the Code

This is performed with a single function call:

```python
# The rank determines the size of the compressed core tensor
core, factors = tucker(transaction_tensor_tl, rank=[2, 2, 2])
```

  * `rank=[2, 2, 2]`: We are telling Tucker to find the **2 most dominant archetypal groups** for senders, the 2 most for receivers, and the 2 most for time.
  * `core`: This is our small summary tensor `G`. Its shape `(2, 2, 2)` shows it's a compressed version of the original.
  * `factors`: This is the list containing our factor matrices `A`, `B`, and `C`. The sender factor matrix, for example, will have a shape of `(10, 2)`, mapping our 10 original senders to the 2 archetypal sender groups.

**Use Case**: Tucker is excellent for denoising data and getting a high-level, structural overview of the transaction network.

-----

## 3\. PARAFAC/CP Decomposition (Extracting Specific Patterns)

### Mathematical Theory

PARAFAC, also known as Canonical Polyadic (CP) decomposition, is a more direct and often more interpretable method. It breaks down the tensor into a sum of a fixed number of **rank-1 tensors**.

A rank-1 tensor is simply the outer product of three vectors. So, for a `rank=R` decomposition, the formula is:
$X \approx \sum_{r=1}^{R} \lambda_r \cdot (\mathbf{a}_r \circ \mathbf{b}_r \circ \mathbf{c}_r)$

Where:

  * $R$ is the number of patterns we want to find.
  * For each pattern $r$, we get three vectors: $\mathbf{a}_r$ (sender involvement), $\mathbf{b}_r$ (receiver involvement), and $\mathbf{c}_r$ (time involvement).
  * $\lambda_r$ is a weight that indicates the overall importance of pattern $r$.

The key advantage here is that each component is a directly readable **"who → whom → when"** pattern.

### How it Relates to the Code

The implementation is also a single line, but the interpretation is different:

```python
# We use rank=2 because we want to find the two patterns we created.
weights, factors = parafac(transaction_tensor_tl, rank=2, ...)
sender_factors, receiver_factors, timestep_factors = factors
```

  * `rank=2`: We are explicitly telling the model to find the **2 most significant, distinct transaction behaviors** in the data.
  * `factors`: This is a list of matrices. `sender_factors` has a shape of `(10, 2)`. **Crucially, the first column is the vector $\mathbf{a}_1$ (sender part of Pattern 1), and the second column is the vector $\mathbf{a}_2$ (sender part of Pattern 2).**
  * The output is a set of vectors that directly describe the patterns, making it perfect for identifying specific illicit activities.

### Interpreting the Results

To find the epicenter of a pattern, we simply find the element with the highest value in each corresponding factor vector.

```python
# Find the index of the max value in the first column (Pattern 1)
dominant_sender = np.argmax(sender_factors[:, 0])
```

This is how the script successfully pinpoints the planted patterns. It looks at the vectors for the first component and finds that Sender 9, Receiver 9, and Timestep 3 have the highest scores, perfectly matching our "illicit money drop" pattern.

In [1]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import tucker, parafac

# --- 1. Create a Mock Dataset ---
# We'll create a small world with 10 senders, 10 receivers, and 5 timesteps.
num_senders = 10
num_receivers = 10
num_timesteps = 5

# Create an empty 3D tensor (our "data cube")
# We'll use floats to represent transaction amounts.
tensor = np.zeros((num_senders, num_receivers, num_timesteps))

print("--- 🔬 Creating a Mock Dataset with Planted Patterns ---")

# Plant Pattern 1: An "Illicit Money Drop"
# Two suspicious senders (8 and 9) send large amounts to one receiver (9)
# specifically at timestep 3. This is a coordinated, hidden pattern.
print("Planting illicit pattern: Senders 8, 9 -> Receiver 9 @ Time 3")
tensor[8, 9, 3] = 100.0  # Sender 8 -> Receiver 9
tensor[9, 9, 3] = 120.0  # Sender 9 -> Receiver 9

# Plant Pattern 2: A "Licit Exchange"
# A major exchange (Sender 0) sends regular, smaller amounts to many
# different receivers across all timesteps. This is a normal, noisy pattern.
print("Planting licit pattern: Sender 0 -> Various Receivers @ All Times")
tensor[0, 0:5, :] = 15.0  # Sender 0 -> Receivers 0-4 at all times

# Add some random noise to make it realistic
tensor += np.random.rand(num_senders, num_receivers, num_timesteps) * 2.0

# Convert to a TensorLy tensor object for our analysis
transaction_tensor_tl = tl.tensor(tensor)
print(f"\nMock tensor created with shape: {transaction_tensor_tl.shape}\n")


# --- 2. Tucker Decomposition (The High-Level Summary) ---
# Tucker finds the main "ingredients" of the data. It compresses the tensor
# into a smaller core and factor matrices representing archetypal groups.
print("---  compressing with Tucker Decomposition ---")
core, factors = tucker(transaction_tensor_tl, rank=[2, 2, 2])
print(f"Shape of compressed core tensor: {core.shape}")
print("Tucker has summarized the data into its 2 most dominant sender-groups, receiver-groups, and time-groups.\n")


# --- 3. PARAFAC/CP Decomposition (Finding Specific Patterns) ---
# PARAFAC is designed to extract a specific number of distinct patterns.
# Since we planted 2 patterns, we ask it to find 2.
print("--- 🔎 Extracting Patterns with PARAFAC Decomposition ---")
# We use rank=2 because we want to find the two patterns we created.
weights, factors = parafac(transaction_tensor_tl, rank=2, init='random', random_state=42)
sender_factors, receiver_factors, timestep_factors = factors


# --- 4. Interpreting the Discovered Patterns ---
print("\n--- ✅ Interpretation of Results ---")
# The algorithm doesn't know which pattern is which, so we check both.
for i in range(2):
    print(f"\n--- Analyzing Discovered Pattern #{i+1} ---")

    # Find the sender, receiver, and timestep most involved in this pattern
    dominant_sender = np.argmax(sender_factors[:, i])
    dominant_receiver = np.argmax(receiver_factors[:, i])
    dominant_timestep = np.argmax(timestep_factors[:, i])

    print(f"Most Dominant Sender: {dominant_sender}")
    print(f"Most Dominant Receiver: {dominant_receiver}")
    print(f"Most Dominant Timestep: {dominant_timestep}")

    # Check if the discovered pattern matches one we planted
    if dominant_sender in [8, 9] and dominant_receiver == 9 and dominant_timestep == 3:
        print("Conclusion: This pattern matches our 'Illicit Money Drop'!")
    elif dominant_sender == 0:
        print("Conclusion: This pattern matches our 'Licit Exchange'!")
    else:
        print("Conclusion: This pattern is a mix of other behaviors.")

--- 🔬 Creating a Mock Dataset with Planted Patterns ---
Planting illicit pattern: Senders 8, 9 -> Receiver 9 @ Time 3
Planting licit pattern: Sender 0 -> Various Receivers @ All Times

Mock tensor created with shape: (10, 10, 5)

---  compressing with Tucker Decomposition ---
Shape of compressed core tensor: (2, 2, 2)
Tucker has summarized the data into its 2 most dominant sender-groups, receiver-groups, and time-groups.

--- 🔎 Extracting Patterns with PARAFAC Decomposition ---

--- ✅ Interpretation of Results ---

--- Analyzing Discovered Pattern #1 ---
Most Dominant Sender: 9
Most Dominant Receiver: 9
Most Dominant Timestep: 3
Conclusion: This pattern matches our 'Illicit Money Drop'!

--- Analyzing Discovered Pattern #2 ---
Most Dominant Sender: 0
Most Dominant Receiver: 3
Most Dominant Timestep: 0
Conclusion: This pattern matches our 'Licit Exchange'!
